In [1]:
pip install ipywidgets transformers


  Using cached ipywidgets-8.1.2-py3-none-any.whl.metadata (2.4 kB)
  Using cached transformers-4.41.1-py3-none-any.whl.metadata (43 kB)
  Using cached widgetsnbextension-4.0.10-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.10-py3-none-any.whl.metadata (4.1 kB)
  Using cached filelock-3.14.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached huggingface_hub-0.23.2-py3-none-any.whl.metadata (12 kB)
  Using cached regex-2024.5.15-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached safetensors-0.4.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
  Using cached fsspec-2024.5.0-py3-none-any.whl.metadata (11 kB)
Using cached ipywidgets-8.1.2-py3-none-any.whl (139 kB)
Using cached transformers-4.41.1-py3-none-any.whl (9.1 MB)
Using 

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from transformers import pipeline

In [ ]:
# Initialize the story generator using a pre-trained model
story_generator = pipeline("text-generation", model="gpt2")

In [ ]:
# Define widgets for user input
genre_input = widgets.Text(
    value='',
    placeholder='Enter genre',
    description='Genre:',
    disabled=False
)

In [ ]:
num_characters_input = widgets.IntText(
    value=1,
    description='Number of Characters:',
    disabled=False
)

reader_age_input = widgets.IntText(
    value=0,
    description='Reader Age:',
    disabled=False
)

character_inputs_container = widgets.VBox()

# Function to create character input fields
def create_character_inputs(num_characters):
    character_inputs_container.children = []
    for i in range(num_characters):
        name_input = widgets.Text(
            placeholder='Enter name',
            description=f'Character {i+1} Name:',
            disabled=False
        )
        sex_input = widgets.Dropdown(
            options=['Male', 'Female', 'Other'],
            value='Male',
            description=f'Character {i+1} Sex:',
            disabled=False
        )
        character_inputs_container.children += (name_input, sex_input)

# Initial character input fields
create_character_inputs(num_characters_input.value)

# Update character input fields when num_characters_input changes
def on_num_characters_input_change(change):
    create_character_inputs(change['new'])

num_characters_input.observe(on_num_characters_input_change, names='value')

# Button to generate the story
generate_button = widgets.Button(
    description='Generate Story',
    disabled=False,
    button_style='',
    tooltip='Click to generate the story',
    icon='check'
)

# Output widget to display the generated story
story_output = widgets.Output()

# Function to handle the button click event
def on_generate_button_click(b):
    genre = genre_input.value
    num_characters = num_characters_input.value
    reader_age = reader_age_input.value

    characters = []
    children = character_inputs_container.children
    for i in range(0, len(children), 2):
        name = children[i].value
        sex = children[i+1].value
        characters.append(f"{name} ({sex})")

    prompt = f"Create a short story in the {genre} genre for a reader aged {reader_age}. "
    prompt += f"The story should have {num_characters} main character(s): "
    prompt += ", ".join(characters) + ".\n\nStory: "

    # Generate the story using the pre-trained model
    story = story_generator(prompt, max_length=300, num_return_sequences=1)[0]['generated_text']

    with story_output:
        clear_output()
        print(story)

generate_button.on_click(on_generate_button_click)

# Display widgets
display(genre_input, num_characters_input, reader_age_input, character_inputs_container, generate_button, story_output)